<a href="https://colab.research.google.com/github/kat-le/cmpe255-unsloth.ai/blob/main/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "unsloth>=2025.1.0" "trl>=0.9.6" "transformers>=4.45.0" "datasets>=2.19.0" \
                "accelerate>=1.0.0" "peft>=0.12.0" "bitsandbytes>=0.44.0"
import os; os.environ["WANDB_DISABLED"] = "true"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [ ]:
import os, torch
from datasets import load_dataset, Dataset, DatasetDict
from unsloth import FastLanguageModel, PatchDPOTrainer, is_bfloat16_supported
from transformers import TrainingArguments
from trl import DPOTrainer

# Model choice: use a 4-bit SFT model Unsloth hosts
MODEL_NAME = "unsloth/zephyr-sft-bnb-4bit"
MAX_SEQ_LEN = 2048
PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

bf16 supported: True
CUDA: NVIDIA A100-SXM4-80GB


In [ ]:
from pathlib import Path
import pandas as pd

DATA_PATH = ""
HF_DATASET = ""

# Fallback: minimal toy sample so the notebook runs
def toy_samples():
    return [
        {"prompt": "Write a friendly greeting for a newsletter.",
         "chosen": "Hey there! Hope your day’s going great—here’s what’s new…",
         "rejected": "Greetings. Below is your requested information."},
        {"prompt": "Explain what DPO is in one sentence.",
         "chosen": "DPO tunes a model to prefer human-preferred responses over rejected ones using pairwise data.",
         "rejected": "DPO is a random algorithm about dogs playing outdoors."},
    ]

# Attempt to load user data
def load_pairwise_dataset():
    if HF_DATASET:
        ds = load_dataset(HF_DATASET, split="train")
        return ds
    if DATA_PATH:
        p = Path(DATA_PATH)
        if p.suffix.lower() == ".jsonl":
            ds = load_dataset("json", data_files={"train": str(p)}, split="train")
            return ds
        elif p.suffix.lower() == ".csv":
            ds = load_dataset("csv", data_files={"train": str(p)}, split="train")
            return ds
        else:
            raise ValueError("DATA_PATH must be a .jsonl or .csv")
    # fallback toy data
    df = pd.DataFrame(toy_samples())
    return Dataset.from_pandas(df)

ds = load_pairwise_dataset()

# Auto-map common column names to the expected DPO ones
CANDIDATE_NAMES = {
    "prompt": ["prompt", "input", "instruction", "question", "messages"],  # 'messages' if you stored chat; see note below
    "chosen": ["chosen", "preferred", "selected", "chosen_response", "answer_good", "output_chosen"],
    "rejected": ["rejected", "unpreferred", "rejected_response", "answer_bad", "output_rejected"],
}

def find_col(target, candidates, cols):
    for c in candidates:
        if c in cols:
            return c
    raise KeyError(f"Could not find a column for `{target}` in {list(cols)}")

cols = set(ds.column_names)
prompt_col = find_col("prompt", CANDIDATE_NAMES["prompt"], cols)
chosen_col = find_col("chosen", CANDIDATE_NAMES["chosen"], cols)
rejected_col = find_col("rejected", CANDIDATE_NAMES["rejected"], cols)

# If your data stores a whole chat turn (list of messages) in `messages`,
# convert to a plain prompt string. This is a simple example; customize as needed.
def msgs_to_prompt(m):
    if isinstance(m, list):
        txt = []
        for turn in m:
            role = turn.get("role", "user")
            content = turn.get("content", "")
            txt.append(f"{role.upper()}: {content}")
        return "\n".join(txt)
    return str(m)

def standardize(example):
    p = example[prompt_col]
    if prompt_col == "messages":
        p = msgs_to_prompt(p)
    return {
        "prompt": str(p),
        "chosen": str(example[chosen_col]),
        "rejected": str(example[rejected_col]),
    }

ds = ds.map(standardize, remove_columns=list(ds.column_names))
ds = ds.shuffle(seed=42)

if len(ds) >= 50:
    dsd = ds.train_test_split(test_size=0.02, seed=42)
    train_ds, eval_ds = dsd["train"], dsd["test"]
else:
    train_ds, eval_ds = ds, None

print(ds[0])
print("Counts - train:", len(train_ds), "eval:", 0 if eval_ds is None else len(eval_ds))


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'prompt': 'Explain what DPO is in one sentence.', 'chosen': 'DPO tunes a model to prefer human-preferred responses over rejected ones using pairwise data.', 'rejected': 'DPO is a random algorithm about dogs playing outdoors.'}
Counts - train: 2 eval: 0


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = MODEL_NAME,
    max_seq_length  = MAX_SEQ_LEN,
    dtype           = None,    # let Unsloth pick (bf16 if supported)
    load_in_4bit    = True,    # QLoRA
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 64,                    # memory/quality knob
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,          # 0 is fastest with Unsloth kernels
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # long context + lower mem
    random_state = 3407,
    max_seq_length = MAX_SEQ_LEN,
)


==((====))==  Unsloth 2025.11.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.11.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

if getattr(model.config, "pad_token_id", None) is None:
    model.config.pad_token_id = tokenizer.pad_token_id

print("pad_token_id:", tokenizer.pad_token_id)

pad_token_id: 0


In [ ]:
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()

args = DPOConfig(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    #num_train_epochs=2,
    max_steps=500,
    logging_steps=5,
    optim="adamw_8bit",
    seed=42,
    fp16=fp16,
    bf16=bf16,
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=args,
    beta=0.1,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    max_length=1024,
    max_prompt_length=512,
    label_pad_token_id=-100,
    padding_value=tokenizer.pad_token_id,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Extracting prompt in train dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Applying chat template to train dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Tokenizing train dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 500 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160 of 7,409,504,256 (2.26% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
5,0.001800,3.292365,-3.104403,1.000000,6.396768,-43.045742,-84.594910,-3.124628,-3.039509,0,0,0
10,0.000200,4.258849,-4.970427,1.000000,9.229276,-33.380905,-103.255165,-3.092798,-3.067286,No Log,No Log,No Log
15,0.000000,5.225806,-6.757623,1.000000,11.983429,-23.711338,-121.127098,-3.002159,-3.014479,No Log,No Log,No Log
20,0.000000,5.726716,-7.831155,1.000000,13.557871,-18.702246,-131.862427,-2.888326,-2.923073,No Log,No Log,No Log
25,0.000000,5.912546,-8.473827,1.000000,14.386373,-16.843937,-138.289154,-2.782449,-2.832193,No Log,No Log,No Log
30,0.000000,5.935572,-8.878456,1.000000,14.814028,-16.613682,-142.335434,-2.706284,-2.773236,No Log,No Log,No Log
35,0.000000,5.831774,-9.148230,1.000000,14.980001,-17.651661,-145.033173,-2.651897,-2.735591,No Log,No Log,No Log
40,0.000000,5.650981,-9.360937,1.000000,15.011917,-19.459591,-147.160248,-2.614627,-2.709079,No Log,No Log,No Log
45,0.000000,5.572520,-9.596250,1.000000,15.168770,-20.244198,-149.513382,-2.589887,-2.690946,No Log,No Log,No Log
50,0.000000,5.661864,-9.821184,1.000000,15.483049,-19.350758,-151.762726,-2.570343,-2.676799,No Log,No Log,No Log


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=500, training_loss=2.0006040215271527e-05, metrics={'train_runtime': 302.4879, 'train_samples_per_second': 26.447, 'train_steps_per_second': 1.653, 'total_flos': 0.0, 'train_loss': 2.0006040215271527e-05, 'epoch': 500.0})

In [ ]:
# Prints the assistant's completion
FastLanguageModel.for_inference(model)

def chat_infer(user_prompt, max_new_tokens=256, do_sample=False, temperature=None):
    messages = [{"role": "user", "content": user_prompt}]

    # 1) Build chat prompt TEXT (not tokenized ids)
    prompt_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,            # <-- return string
    )

    # 2) Tokenize to get input_ids and attention_mask
    enc = tokenizer(
        prompt_text,
        return_tensors="pt",
        add_special_tokens=False,
    )
    enc = {k: v.to(model.device) for k, v in enc.items()}
    input_len = enc["input_ids"].shape[1]

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    if do_sample and temperature is not None:
        gen_kwargs["temperature"] = temperature

    with torch.inference_mode():
        out = model.generate(**enc, **gen_kwargs)

    # Decode ONLY new tokens
    generated = out[0, input_len:]
    text = tokenizer.decode(generated, skip_special_tokens=True)

    for t in ("<|assistant|>", "<|user|>", "[INST]", "[/INST]", "<s>", "</s>"):
        text = text.replace(t, "")
    print(text.strip())

chat_infer("Explain what Direct Preference Optimization (DPO) does in one short paragraph.", do_sample=False)


Direct Preference Optimization (DPO) is a machine learning technique that enables computers to learn and make decisions based on human preferences without explicit feedback. DPO algorithms analyze historical data and identify patterns in user behavior to predict which options will be preferred in the future. By optimizing for these preferences, DPO can improve the relevance and effectiveness of recommendations, search results, and other decision-making processes, ultimately leading to a better user experience.
